In [1]:
import numpy as np
from skimage import io
import napari
import pandas as pd
import zarr
import os
import pickle
import matplotlib.pyplot as plt
import cv2
from cellpose import models
import tiffile as tf
import glob

In [2]:
import pickle 
import pickle 
print(pickle.format_version)

4.0


### Get Annotation data

In [3]:
##### change path
# first path need to be first brain,second path need to be another brain
fix_n5_path = ['/mnt/aperto/fused/fused.n5','/mnt/aperto/fused/fused.n5']
# save path
#create the directory if it does not exist
##### change path
# you need to change to the folder you want
directory = './tatz_annotation/'
if not os.path.exists(directory):
    os.makedirs(directory)
    print("Directory created:", directory)
##### change path
save_path = directory
##### change path
#you need to change to the folder you want metadata path
meta_path = './tatz_annotation/tatz_annotation.pkl'

# create Zarr file object
fix_zarr = [zarr.open(store=zarr.N5Store(fix_n5_path[0]), mode='r'), zarr.open(store=zarr.N5Store(fix_n5_path[1]), mode='r')]
# if you use ngff ome.zarr
# mov_zarr_path = '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/registration/round02.zarr'
# mov_zarr = zarr.open(mov_zarr_path, mode='r')

n5_setups = list(fix_zarr[0].keys())

voxel_size = (2.0,1.3,1.3)

/tmp/ipykernel_1294945/3534847048.py:19: FutureWarning: The N5Store is deprecated and will be removed in a Zarr-Python version 3, see https://github.com/zarr-developers/zarr-python/issues/1274 for more information.
  fix_zarr = [zarr.open(store=zarr.N5Store(fix_n5_path[0]), mode='r'), zarr.open(store=zarr.N5Store(fix_n5_path[1]), mode='r')]


In [4]:
# make metadata file if it does not exist.
if not os.path.exists(meta_path):
    df = pd.DataFrame(columns=['ID', 'integer_ID', 'instance_counts', 'corner', 'source', 'ref_channel', 'channel', 'crop_size', 'isHard', 'plane_position'])
else:
    df = pd.read_pickle(meta_path)


In [5]:
resolusion_0_shape = fix_zarr[0][n5_setups[1]]['timepoint0']['s0'].shape
resolusion_2_shape = fix_zarr[0][n5_setups[3]]['timepoint0']['s2'].shape
scale_size = []
for i in range(len(resolusion_0_shape)):
    scale_size.append(resolusion_2_shape[i]/resolusion_0_shape[i])

In [6]:
# set your parameters here
reference_chan = 3 # Integer or None
segment_chan = 4

# [100,256,256] crop size and FoV [100,768,768] are recommended for the 2D annotation
crop_size = [100,256,256]
FoV = [100,768, 768]
FoV1 = [1, 2137, 1603]
# set True for 2D annotation and set False for 3D annotation
select_plane = True

# processing of parameters
if not all([(j-i)>=0 for i,j in zip(crop_size, FoV)]):
    raise ValueError('FoV should be larger than crop_size')

#### load original images need to label

In [7]:
# find all files in the directory ending with '_img.tif', and '_mask.tif' 
##### change path
#you need to change this path to your image and mask file path
img_dir = '/mnt/aperto/yin/cellpose_training/data/train/' 
img_files = sorted(glob.glob(f'{img_dir}*_img.tif'))
mask_files = sorted(glob.glob(f'{img_dir}*mask.tif'))
img_id = [item.split('/')[-1][:4] for item in img_files]

#### load test dataframe

In [8]:
with open('./val_train.pkl', 'rb') as f:
    val_train_df = pickle.load(f)

In [9]:
# optional run, check source of data
set(val_train_df['source'])

{'/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/fused/fused.n5',
 '/mnt/ampa02_data01/tmurakami/240425_whole_4color_2nd_M037-3pb/fused/fused.n5'}

In [10]:
id_list = val_train_df['ID'].astype(str).tolist()

In [11]:
begin_id = 27

In [ ]:
for ori_id in id_list[begin_id+10:]:
    ori_corner_position = [val_train_df['corner'][int(ori_id)]]
    ori_plane = int(val_train_df['plane_position'][int(ori_id)])
    segment_chan = int(val_train_df['channel'][int(ori_id)])
    reference_chan =  int(val_train_df['ref_channel'][int(ori_id)])
    #check which source it should be int 0 or 1
    brain_source = int(val_train_df['source'][int(ori_id)].split('/')[-3].split('_')[-2][0])-1
    
    pos = ori_corner_position[0]
    if len(pos) <= 2:
        raise ValueError('The position should have length 3')
    elif len(pos) == 3:
        isHard = 0
    elif len(pos) == 4:
        isHard = pos[-1]
        pos = pos[:-1]
    else:
        raise ValueError('You have a wrong position format')
        
    print(f"The index {ori_id} with the position {pos}")
    idx = int(ori_id)

    z_index = int((pos[0]+ori_plane)*scale_size[0])
    # prepare to make border lines
    a = [1, pos[1]*scale_size[1], pos[2]*scale_size[2]]
    b = [1, (pos[1]+256)*scale_size[1], (pos[2]+256)*scale_size[2]]
    top_border_corner = tuple(a)
    bottom_border_corner = tuple(b)
   
    #### full brain plane
    zarr_array = fix_zarr[brain_source][n5_setups[segment_chan]]['timepoint0']['s2']
    selected_plane_1 = zarr_array[z_index, :, :]
    zarr_array = fix_zarr[brain_source][n5_setups[reference_chan]]['timepoint0']['s2']
    selected_plane_2 = zarr_array[z_index, :, :]
    #blue_image = np.full_like(zarr_array[0, :, :], fill_value=255)
    viewer1 = napari.Viewer()
    
    @viewer1.bind_key('q')
    def close_viewer(viewer):
        print("Closing viewer...")
        viewer.close()

     #set hide hotkey
    @viewer1.bind_key('h')
    def toggle_layer_visibility(viewer):
        layer = viewer.layers.selection.active
        if layer is not None:
            layer.visible = not layer.visible
    
    #data3 = np.zeros(shape, dtype=np.uint8)
    # Initialize the first Napari viewer and add the first Z-plane
   
    viewer1.add_image(selected_plane_1, name=f'Z-plane {ori_plane}', colormap='gray',opacity = 0.5)

    viewer1.add_image(selected_plane_2, name=f'Z-plane {ori_plane}', colormap='gray',opacity = 0.5)
    #viewer1.add_image(blue_image, name='Cover Layer')
    viewer1.add_shapes([[bottom_border_corner[1],bottom_border_corner[2]],[top_border_corner[1],bottom_border_corner[2]]],
                      edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer1.add_shapes([[top_border_corner[1],bottom_border_corner[2]],[top_border_corner[1],top_border_corner[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer1.add_shapes([[bottom_border_corner[1],bottom_border_corner[2]],[bottom_border_corner[1],top_border_corner[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer1.add_shapes([[bottom_border_corner[1],top_border_corner[2]],[top_border_corner[1],top_border_corner[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line') 
 
    # find out any duplication between the current data and the metadata
    # if it is duplicated, ask 
    flag = False
    if df['corner'].isin([pos]).any():
        for k in df['integer_ID'][df['corner'].isin([pos])].to_list():
            if ((df.loc[k,'source'] == fix_n5_path[brain_source]) and 
                (df.loc[k,'ref_channel'] == reference_chan) and 
                (df.loc[k,'channel'] == segment_chan) and 
                (df.loc[k,'crop_size'] == crop_size) and
                (df.loc[k,'select_plane'] == select_plane)):
                flag = True
                idx = k
    if flag:
        ans = input("Do you want to re-analyze the data? y or n")
        if ans != 'y':
            continue
        
    # set file path to be saved for both image and mask
    prefix = str(idx)
    while len(prefix) < 4:
        prefix = '0' + prefix
    img_path = os.path.join(save_path, prefix+'_img.tif')
    mask_path = os.path.join(save_path, prefix+'_mask.tif')
    # get the image of a channel to be segmented
    FoV_stack = []
    img = fix_zarr[brain_source][n5_setups[segment_chan]]['timepoint0']['s0']

    # set the corner of FoV in napari
    top_corner = tuple(i-(k-j)//2 for i,j,k in zip(pos, crop_size, FoV))
    bottom_corner = tuple(i+j+(k-j)//2 for i,j,k in zip(pos, crop_size, FoV))
    top_corner = tuple(j if j>=i else i for i,j in zip([0,0,0],top_corner))
    bottom_corner = tuple(j if j<=i else i for i,j in zip(img.shape,bottom_corner))
    
    # prepare to make border lines
    top_border_corner = tuple((k-j)//2 for j,k in zip(crop_size, FoV))
    bottom_border_corner = tuple(j+(k-j)//2 for j,k in zip(crop_size, FoV))
    
    FoV_segment = img[tuple(slice(i,j) for i,j in zip(top_corner, bottom_corner))]
   
    # get the image of a reference of channel
    if reference_chan is not None:
        img = fix_zarr[brain_source][n5_setups[reference_chan]]['timepoint0']['s0']
        FoV_reference = img[tuple(slice(i,j) for i,j in zip(top_corner, bottom_corner))]
        FoV_stack.append(FoV_reference)
    
    FoV_stack.append(FoV_segment)
    FoV_stack = np.stack(FoV_stack)

    ##### FoV is a 2channel(reference and signal) Field of view
    # open Napari. Pause for loop until close the window

    ## add label layer data
    shape = (100, 768, 768)
    data = np.zeros(shape, dtype=np.uint8)
    viewer = napari.Viewer()

    @viewer.bind_key('q')
    def close_viewer(viewer):
        print("Closing viewer...")
        viewer.close()

     #set hide hotkey
    @viewer.bind_key('h')
    def toggle_layer_visibility(viewer):
        layer = viewer.layers.selection.active
        if layer is not None:
            layer.visible = not layer.visible

    viewer.add_image(FoV_stack, channel_axis=0, scale=voxel_size, contrast_limits=[0,65535])
    viewer.add_shapes([[bottom_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]],[top_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]]],
                      edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer.add_shapes([[top_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]],[top_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer.add_shapes([[bottom_border_corner[1]*voxel_size[1],bottom_border_corner[2]*voxel_size[2]],[bottom_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line')
    viewer.add_shapes([[bottom_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]],[top_border_corner[1]*voxel_size[1],top_border_corner[2]*voxel_size[2]]],
                  edge_width=2,edge_color='white',ndim=2,shape_type='line') 

    labels_class1 = viewer.add_labels(data,  name=f'Label({ori_plane})', scale=voxel_size)
    labels_class1.opacity = 1.0
    labels_class1.brush_size = 1

    viewer.camera.zoom = 1.5
    viewer.dims.current_step = (ori_plane,400,400)
    
    viewer.show(block=True)
    
    sub_area_slicer = tuple(slice(i,j) for i,j in zip(top_border_corner,bottom_border_corner))
    ######
    # subarea shape
    # save images and segmentation.
    img = np.swapaxes(FoV_stack[(slice(0,None),)+sub_area_slicer],0,1)
    ######
    # modify this to cellpose label
    #labels_img = labels.data[sub_area_slicer]
    labels_img_class1 = labels_class1.data[sub_area_slicer]
    
    if select_plane:
        print('selected and save')
        ##########
        # modify this to original label
        plane_pos = np.argmax((labels_class1.data>0).sum(axis=(1,2)))
        img = img[plane_pos,...]
        labels_img_class1 = labels_img_class1[plane_pos,...]
        io.imsave(img_path, img, plugin='tifffile', imagej=True, metadata={'axes': 'CYX'})
        io.imsave(mask_path, labels_img_class1, plugin='tifffile')
        

    else:
        print('#############')
        print('not selected and save')
        io.imsave(img_path, img, plugin='tifffile', imagej=True, metadata={'axes': 'ZCYX'})

    # update the metadata
    df.loc[idx,'ID'] = prefix
    df.loc[idx,'integer_ID'] = idx
    count = (np.unique(labels_img_class1)).size - 1
    df.loc[idx,'instance_counts'] = count
    df.at[idx,'corner'] = pos
    df.loc[idx, 'source'] = fix_n5_path[brain_source]
    df.loc[idx, 'ref_channel'] = reference_chan
    df.loc[idx, 'channel'] = segment_chan
    df.at[idx, 'crop_size'] = crop_size
    df.loc[idx, 'select_plane'] = select_plane
    df.loc[idx, 'isHard'] = isHard
    if select_plane:
        df.loc[idx, 'plane_position'] = int(plane_pos)
    else:
        df.loc[idx, 'plane_position'] = -1


The index 0221 with the position [1197, 3800, 4884]


In [13]:
df.to_pickle(meta_path)

In [12]:
 id_list.index('0158')

27

In [14]:
# if you don't want to run all again, set the beginning id as begin
curent_id = id_list.index(ori_id)
begin_id = curent_id
print(f'current image id {curent_id}, next begin id {begin_id}')

current image id 46, next begin id 46


In [20]:
# (optional)if you want to start all over again, reset begin_id as 0
# begin_id = 0